In [16]:
# %matplotlib notebook
from IPython.display import display, IFrame, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))

import numpy             as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

from matplotlib          import rc 
from matplotlib          import cm

import pandas as pd
import corner

import os
import os.path as path
import wget

import emcee

from tqdm import tqdm

from scipy.integrate import dblquad
from scipy.integrate import quad

In [17]:
if not path.exists('Data'):
    os.mkdir('Data')
    
file = 'Data/Gaussiano.csv'
url ='https://raw.githubusercontent.com/asegura4488/Database/main/MetodosComputacionalesReforma/Gaussiano.csv'

if not path.exists(file):
    Path_ = wget.download(url,file)
    
else:
    Path_ = file

In [18]:
data = pd.read_csv(Path_)
data

,x
0,0.906451
1,5.446117
2,6.840238
3,0.743644
4,7.338518
...,...
95,1.497647
96,4.236658
97,4.678540
98,6.479197


In [19]:
X = data.x

In [20]:
def prior(mu, sigma):
    if 3 <= mu <= 5 and 0.5 <= sigma <= 3.5:
        return 1
    else:
        return 0

In [21]:
def LogPrior(mu, sigma):
    
    if 3. <= mu <= 5. and 0.5 <= sigma <= 3.5:
        return 0.
    else:
        return -np.inf

In [22]:
def Gauss(mu, sigma, x):
    return np.exp( -0.5*(x-mu)**2/sigma**2  )/np.sqrt(2*np.pi*sigma**2)

In [23]:
def Likelihood(mu, sigma, x):
    return Gauss(mu,sigma,x)

In [24]:
def JointLikelihood(mu, sigma, x):
    return np.sum(np.log(Likelihood(mu, sigma, x)))

In [25]:
def LogPosterior(mu, sigma, x):
    LogP = LogPrior(mu, sigma)
    if not np.isfinite(LogP):
        return -np.inf
    else:
        return JointLikelihood(mu, sigma, x) + LogP

In [26]:
def metropolis(mu, sigma, X,  NSteps=int(2e4 + 1), delta= 0.4):
    x = np.zeros((NSteps,2))
    x[0, 0] = mu
    x[0, 1] = sigma
    
    for i in tqdm(range(1,NSteps)):
        P0 = LogPosterior(x[i-1, 0], x[i-1, 1], X)
        mu_ = x[i-1, 0] + delta*2*(np.random.rand()-0.5)
        sigma_ = x[i-1, 1] + delta*2*(np.random.rand()-0.5)
        
        P_ = LogPosterior(mu_, sigma_, X)
        
        if P_ == -np.inf:
            alpha = P_
        else:
            alpha = np.minimum( 1, P_/P0 )
        
        g = np.random.rand()
        
        if alpha > g:
            x[i, 0] = mu_
            x[i, 1] = sigma_
        else:
            x[i,:] = x[i-1,:]
            
    return x[1000:,:]

In [27]:
samples = metropolis(4.5, 1.5, X, )

100%|██████████████████████████████████████████████████████████████████████████| 20000/20000 [00:19<00:00, 1004.89it/s]


In [29]:
Best_mu = np.percentile(samples[:, 0], 50)
Best_mu

4.008428646683392

In [30]:
inf_mu = np.percentile(samples[:, 0], 16)
inf_mu

3.293141659789672

In [32]:
sup_mu = np.percentile(samples[:, 0], 84)
sup_mu

4.7036745015211014

$\hat{\mu} = 4^{3.3}_{4} at \%68 CL$ 

In [36]:
Best_sigma = np.percentile(samples[:, 1], 50)
Best_sigma

1.8046678527734181

In [37]:
inf_sigma = np.percentile(samples[:, 1], 16)
inf_sigma

0.7211199816119844

In [39]:
sup_sigma = np.percentile(samples[:, 1], 84)
sup_sigma

2.983398935447275

$\hat{\sigma} = 1.8^{0.7}_{3} at \%68 CL$ 